In [1]:
import pandas as pd

data_directory = '../data/'

# here is the corrected data
filename = "corrected_data.json"

# take the transpose so that the column names are the metadata keys, not the story ids
df = pd.read_json(data_directory + filename).transpose()

# let's see what it looks like!
df.head()

,author_id,characters,genres,language,num_chapters,num_favs,num_follows,num_reviews,num_words,published,rated,status,title,updated
10000036,2846408,"[[Ron W., Hermione G.]]","[Romance, Humor]",English,1,5,2,3,5436,1389050085,T,Complete,Kilts and other adversities,1389050085
10000109,5232542,"[James P., Lily Evans P.]","[Humor, Romance]",English,1,4,NaN,2,747,1389051817,K+,Complete,Of Dead Puppies and Dropped Pianos,1389051817
10000111,5438139,[],[],Spanish,1,7,6,5,400,1389051884,K+,Incomplete,Querido primo Harry,1389051884
10000114,5437478,"[Hermione G., Draco M.]","[Romance, Humor]",Spanish,1,11,13,8,1757,1389051911,K+,Incomplete,Conciertos en Hogwarts,1389051911
10000137,4626918,"[Bellatrix L., Luna L.]","[Humor, Horror]",English,3,NaN,NaN,3,659,1389052244,T,Complete,Crazy songs from crazy people,1389414254


In [103]:
dev = df.head(n=1000).copy()
len(dev)

1000

In [133]:
dev = df.copy()

In [134]:
characters = dev[dev['characters'].apply(lambda x: len(x)>0)]
len(characters)

507475

In [4]:
def isListOfLists(obj):
    return any(isinstance(el, list) for el in obj)

In [5]:
def flatten_list(l):
    if isListOfLists(l):
        returnList = []
        for sublist in l:
            if isinstance(sublist, list):
                for item in sublist:
                    returnList.append(item)
            else:
                returnList.append(sublist)
        return returnList;
    else:
        return l

In [74]:
test1 = [['hi', 'there']]
test2 = ['hi', 'there']
test3 = [['hi', 'there'],'joe']
test4 = [['hi', 'there'],'joe',['seeya','later']]
print(flatten_list(test1))
print(flatten_list(test2))
print(flatten_list(test3))
print(flatten_list(test4))

['hi', 'there']
['hi', 'there']
['hi', 'there', 'joe']
['hi', 'there', 'joe', 'seeya', 'later']


In [135]:
flattened = characters['characters'].map(lambda l: flatten_list(l))

In [136]:
# from https://stackoverflow.com/questions/29034928/pandas-convert-a-column-of-list-to-dummies
dummies = pd.get_dummies(flattened.apply(pd.Series).stack()).sum(level=0)

In [137]:
# from https://stackoverflow.com/questions/20574257/constructing-a-co-occurrence-matrix-in-python-pandas
dummies_asint = dummies.astype(int)
coocc = dummies_asint.T.dot(dummies_asint)

In [138]:
coocc['Harry P.']['Harry P.']

151951

In [139]:
coocc.head()

,,A. Dippet,A. Kirke,A. Lynch,A. Pye,A. Sinistra,Aberforth D.,Abraxas M.,Adrian P.,Alastor M.,...,Whomping Willow,William S.,William the Pukwudgie,Winky,Xenophilius L.,Yaxley,Zacharias S.,Zacharias S.] Megan J.,Zhang Fei,Úrsula F.
,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A. Dippet,0,24,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
A. Kirke,0,0,11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A. Lynch,0,0,0,13,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A. Pye,0,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
coocc.drop(coocc.keys()[0], 1, inplace=True)
coocc.head()

,A. Dippet,A. Kirke,A. Lynch,A. Pye,A. Sinistra,Aberforth D.,Abraxas M.,Adrian P.,Alastor M.,Albert R.,...,Whomping Willow,William S.,William the Pukwudgie,Winky,Xenophilius L.,Yaxley,Zacharias S.,Zacharias S.] Megan J.,Zhang Fei,Úrsula F.
,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A. Dippet,24,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
A. Kirke,0,11,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A. Lynch,0,0,13,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A. Pye,0,0,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [140]:
coocc.keys()

Index(['', 'A. Dippet', 'A. Kirke', 'A. Lynch', 'A. Pye', 'A. Sinistra',
       'Aberforth D.', 'Abraxas M.', 'Adrian P.', 'Alastor M.',
       ...
       'Whomping Willow', 'William S.', 'William the Pukwudgie', 'Winky',
       'Xenophilius L.', 'Yaxley', 'Zacharias S.', 'Zacharias S.] Megan J.',
       'Zhang Fei', 'Úrsula F.'],
      dtype='object', length=414)

In [142]:
for name in coocc.keys():
    print("{0}:{1}".format(name,coocc[name][name]))

:1
A. Dippet:24
A. Kirke:11
A. Lynch:13
A. Pye:3
A. Sinistra:175
Aberforth D.:317
Abraxas M.:245
Adrian P.:222
Alastor M.:654
Albert R.:2
Albus D.:10372
Albus S. P.:6693
Alecto C.:129
Alexia B.:4
Alice L.:988
Alicia S.:422
Alphard B.:98
Ambrosius F.:12
Amelia B.:207
Amos D.:44
Amy B.:38
Amycus C.:76
Andromeda T.:2658
Angelina J.:1735
Anne P.:54
Anne P.]:1
Anthony G.:155
Antioch P.:51
Antonin D.:140
Arabella F.:124
Aragog:9
Arcturus B.:43
Argus F.:474
Ariana D.:279
Arthur W.:1487
Astoria G.:2357
Audrey W.:374
Augusta L.:133
Augustus R.:43
Avery:49
Bane:31
Barnabas Cuffe:3
Barty C. Jr.:695
Barty C. Sr.:103
Basilisk:65
Bathilda B.:33
Bellatrix L.:7242
Belvina B.:9
Benjy F.:65
Bertha J.:19
Bill W.:1808
Blaise Z.:3975
Bloody Baron:152
Bob H.:21
Buckbeak:39
C. Binns:62
C. Fudge:159
C. Warrington:57
Cadmus P.:47
Cadwallader:9
Callidora B.:18
Caradoc D.:53
Caspar C.:2
Cassiopeia B.:42
Cedrella B.:67
Cedric D.:2426
Chadwick B.:2
Charis B.:11
Charity B.:69
Charlie W.:2655
Charlus P.:87
Cho C.:22

In [173]:
names_to_delete = []
names_to_keep = []
for name in coocc.keys():
    if coocc[name][name] < 800:
        names_to_delete.append(name)
    else:
        names_to_keep.append(name)
print(len(names_to_delete))
print(len(names_to_keep))
names_to_keep

347
67


['Albus D.',
 'Albus S. P.',
 'Alice L.',
 'Andromeda T.',
 'Angelina J.',
 'Arthur W.',
 'Astoria G.',
 'Bellatrix L.',
 'Bill W.',
 'Blaise Z.',
 'Cedric D.',
 'Charlie W.',
 'Cho C.',
 'Daphne G.',
 'Dean T.',
 'Dominique W.',
 'Draco M.',
 'Dudley D.',
 'Fleur D.',
 'Fred W.',
 'Gellert G.',
 'George W.',
 'Ginny W.',
 'Godric G.',
 'Hannah A.',
 'Harry P.',
 'Hermione G.',
 'Hugo W.',
 'James P.',
 'James S. P.',
 'Katie B.',
 'Lavender B.',
 'Lily Evans P.',
 'Lily Luna P.',
 'Lucius M.',
 'Lucy W.',
 'Luna L.',
 'Lysander S.',
 'Marauders',
 'Marlene M.',
 'Minerva M.',
 'Molly W.',
 'N. Tonks',
 'Narcissa M.',
 'Neville L.',
 'OC',
 'Oliver W.',
 'Pansy P.',
 'Percy W.',
 'Peter P.',
 'Petunia D.',
 'Regulus B.',
 'Remus L.',
 'Rodolphus L.',
 'Ron W.',
 'Rose W.',
 'Salazar S.',
 'Scorpius M.',
 'Seamus F.',
 'Severus S.',
 'Sirius B.',
 'Teddy L.',
 'Theodore N.',
 'Tom R. Jr.',
 'Victoire W.',
 'Viktor K.',
 'Voldemort']

In [176]:
coocc['OC'].nlargest(5)

OC            32822
Harry P.       6345
Sirius B.      4161
Draco M.       4130
Severus S.     2816
Name: OC, dtype: int64

In [198]:
coocc.to_csv(data_directory+'cooccurrence.csv')

In [192]:
char_freqs = []
for name in coocc.keys():
#    if(coocc[name][name]>1):
    char_freqs.append([name,coocc[name][name]])
char_freq_df = pd.DataFrame(char_freqs, columns=['name','count'])
char_freq_df.head()

,name,count
0,,1
1,A. Dippet,24
2,A. Kirke,11
3,A. Lynch,13
4,A. Pye,3


In [263]:
sorted_char_freq = char_freq_df.sort_values(['count','name'], ascending=False)

In [264]:
top_200 = sorted_char_freq.head(n=200)

In [265]:
top_200.head(n=100)

,name,count
162,Harry P.,151951
171,Hermione G.,127244
98,Draco M.,110555
366,Severus S.,50641
220,Lily Evans P.,46677
196,James P.,45143
367,Sirius B.,44733
152,Ginny W.,44599
344,Ron W.,38037
333,Remus L.,37010


In [266]:
names = top_200.pop('name')
top_200.index = names
top_200.head()

,count
name,
Harry P.,151951
Hermione G.,127244
Draco M.,110555
Severus S.,50641
Lily Evans P.,46677


In [267]:
top_200['count']['Remus L.']

37010

In [268]:
marauders_bonus = top_200['count']['Marauders']
marauders = ['Remus L.', 'Sirius B.', 'James P.', 'Peter P.']
for marauder in marauders:
    orig_val = top_200['count'][marauder]
    top_200.set_value(marauder, 'count', orig_val + marauders_bonus)

In [269]:
founders_bonus = top_200['count']['Founders']
print(founders_bonus)
founders = ['Godric G.', 'Salazar S.', 'Helga H.', 'Rowena R.']
for founder in founders:
    orig_val = top_200['count'][founder]
    print(orig_val)
    top_200.set_value(founder, 'count', orig_val + founders_bonus)

94
843
1300
559
790


In [270]:
tom_riddle_count = top_200['count']['Tom R. Jr.']
voldemort_count = top_200['count']['Voldemort']
top_200.set_value('Voldemort', 'count', tom_riddle_count+ voldemort_count)

,count
name,
Harry P.,151951
Hermione G.,127244
Draco M.,110555
Severus S.,50641
Lily Evans P.,46677
James P.,47897
Sirius B.,47487
Ginny W.,44599
Ron W.,38037


In [272]:
top = top_200.drop(['Marauders', 'Founders', 'Tom R. Jr.'])

In [273]:
len(top)

197

In [274]:
top = top.sort_values('count', ascending=False)

In [275]:
percents = top/top.sum()

In [276]:
percents.columns=['percent']

In [277]:
percents

,percent
name,
Harry P.,0.152031
Hermione G.,0.127311
Draco M.,0.110613
Severus S.,0.050668
James P.,0.047922
Sirius B.,0.047512
Lily Evans P.,0.046701
Ginny W.,0.044622
Remus L.,0.039785


In [278]:
percents.to_csv(data_directory+'char_freq_ff.csv')